In [1]:
import tensorflow as tf
from pathlib import Path

tf.get_logger().setLevel("ERROR")
pwd = Path.cwd()

2021-08-15 11:28:14.122764: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-15 11:28:14.122791: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
tf.__version__

'2.5.1'

**(?)** Why My GPU not working for TF2.5.1?

In [3]:
import tensorflow_datasets as tfds

# pt_to_en means Portuguese to English
examples, metadata = tfds.load(
    "ted_hrlr_translate/pt_to_en",
    with_info=True,
    as_supervised=True,
)

type(examples), type(metadata)

2021-08-15 11:33:43.145529: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "Not found: Could not locate the credentials file.". Retrieving token from GCE failed with "Failed precondition: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

...

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling ted_hrlr_translate-test.tfrecord...:   0%|          | 0/1803 [00:00<?, ? examples/s]

Dataset ted_hrlr_translate downloaded and prepared to /home/phunc20/tensorflow_datasets/ted_hrlr_translate/pt_to_en/1.0.0. Subsequent calls will reuse this data.


2021-08-15 11:34:41.356919: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-08-15 11:34:41.394975: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-15 11:34:41.395764: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1070 with Max-Q Design computeCapability: 6.1
coreClock: 1.379GHz coreCount: 16 deviceMemorySize: 7.93GiB deviceMemoryBandwidth: 238.66GiB/s
2021-08-15 11:34:41.395864: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-15 11:34:41.395946: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not 

(dict, tensorflow_datasets.core.dataset_info.DatasetInfo)

In [4]:
examples.keys()

dict_keys([Split('train'), Split('validation'), Split('test')])

In [5]:
train_examples, val_examples = examples["train"], examples["validation"]
type(train_examples), type(val_examples)

(tensorflow.python.data.ops.dataset_ops.PrefetchDataset,
 tensorflow.python.data.ops.dataset_ops.PrefetchDataset)

In [9]:
for pt, en in train_examples.take(1):
    print(f"type(pt) = {type(pt)}")
    print(f"pt = {pt}")
    print(f"pt.numpy() = {pt.numpy()}")
    print("Portuguese: ", pt.numpy().decode("utf-8"))
    print()
    print(f"en = {en}")
    print(f"en.numpy() =\n{en.numpy()}")
    print("English: ", en.numpy().decode("utf-8"))

type(pt) = <class 'tensorflow.python.framework.ops.EagerTensor'>
pt = b'e quando melhoramos a procura , tiramos a \xc3\xbanica vantagem da impress\xc3\xa3o , que \xc3\xa9 a serendipidade .'
pt.numpy() = b'e quando melhoramos a procura , tiramos a \xc3\xbanica vantagem da impress\xc3\xa3o , que \xc3\xa9 a serendipidade .'
Portuguese:  e quando melhoramos a procura , tiramos a única vantagem da impressão , que é a serendipidade .

en = b'and when you improve searchability , you actually take away the one advantage of print , which is serendipity .'
en.numpy() =
b'and when you improve searchability , you actually take away the one advantage of print , which is serendipity .'
English:  and when you improve searchability , you actually take away the one advantage of print , which is serendipity .


2021-08-15 11:48:40.919669: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [14]:
train_en = train_examples.map(lambda pt, en: en)
train_pt = train_examples.map(lambda pt, en: pt)
print(f"train_en = {train_en}")
print(f"train_en.take(1) = {train_en.take(1)}")
print(f"train_pt.take(1) = {train_pt.take(1)}")

#print(f"train_en.take(1)[0] = {train_en.take(1)[0]}")
## TypeError: 'TakeDataset' object is not subscriptable

#print(f"next(train_en.take(1)) = {next(train_en.take(1))}")
## TypeError: 'TakeDataset' object is not an iterator


train_en = <MapDataset shapes: (), types: tf.string>
train_en.take(1) = <TakeDataset shapes: (), types: tf.string>
train_pt.take(1) = <TakeDataset shapes: (), types: tf.string>


In [15]:
from tensorflow_text.tools.wordpiece_vocab import bert_vocab_from_dataset as bert_vocab

bert_tokenizer_params = dict(lower_case=True)


`dict(lower_case=True)` is just initializing a dictionary with one pair of key-value
```python
"lower_case": True
```
Cf. below

In [1]:
dict(are_you_kidding=True, what_time_is_it="14h33")

{'are_you_kidding': True, 'what_time_is_it': '14h33'}